In [1]:
%matplotlib widget
from nilearn_tools import compute_connectome, subjectList
from acme import ParallelMap, esi_cluster_setup

In [2]:
myClient = esi_cluster_setup(n_workers=10, partition="8GBXS")

/opt/conda/envs/esi-2023a/lib/python3.10/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35287 instead
  warnings.warn(
<slurm_cluster_setup> SLURM workers ready: 10/10 	[elapsed time 00:25 | timeout at 01:00]
ACME - INFO -  Parallel computing client ready, dashboard accessible at http://10.100.32.17:35287/status


In [3]:
with ParallelMap(compute_connectome, subjectList, result_shape=(264, 264, None)) as pmap:
    results = pmap.compute()

ACME > ANNOUNCE <  This is ACME v. 2023.4
ACME - INFO -  Attaching to parallel computing client <Client: 'tcp://10.100.32.17:35271' processes=10 threads=10, memory=74.50 GiB>
ACME - INFO -  Preparing 21 parallel calls of `compute_connectome` using 10 workers
100% |██████████| 21/21 [01:36<00:00]
ACME > ANNOUNCE <  SUCCESS!
ACME - INFO -  Results have been saved to /cs/home/fuertingers/ACME_20230713-102727-129102/compute_connectome.h5
